In [3]:
#Array processing
import numpy as np

#Data analysis, wrangling and common exploratory operations
import pandas as pd
from pandas import Series, DataFrame
from IPython.display import display

#For visualization. Matplotlib for basic viz and seaborn for more stylish figures + statistical figures not in MPL.
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp
#SVD for Sparse matrices
from scipy.sparse.linalg import svds

from sklearn.metrics.pairwise import euclidean_distances

from collections import defaultdict, Counter
import operator

from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 6)




In [10]:
#Load the user data
users_df = pd.read_csv('ml-100k/u.user', sep='|', names=['UserId', 'Age', 'Gender', 'Occupation', 'ZipCode'])
#Load the movies data: we will only use movie id and title for this assignment
movies_df = pd.read_csv('ml-100k/u.item', sep='|', names=['MovieId', 'Title'], usecols=range(2))

#Load the ratings data: ignore the timestamps
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', names=['UserId', 'MovieId', 'Rating'],usecols=range(3))

movie_ratings_df = pd.merge(movies_df, ratings_df)
movielens_df = pd.merge(movie_ratings_df, users_df)
movielens_df.head()

,MovieId,Title,UserId,Rating,Age,Gender,Occupation,ZipCode
0,1,Toy Story (1995),308,4,60,M,retired,95076
1,4,Get Shorty (1995),308,5,60,M,retired,95076
2,5,Copycat (1995),308,4,60,M,retired,95076
3,7,Twelve Monkeys (1995),308,4,60,M,retired,95076
4,8,Babe (1995),308,5,60,M,retired,95076


In [34]:
#Exploratory Analysis
#Task t1a: Print the NAME of the top-10 movies with most ratings
print movielens_df.groupby('Title').size().order(ascending=False)[:10]

Title
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
dtype: int64


In [22]:
#Task t1b: Let us now analyze the rating behavior of the 1000 users in the dataset
#  Create a histogram based on the number of ratings per user with 50 bins. 
# Title="Count of Ratings per User", XLabel="Ratings per user", YLabel="#Users"

#t1b_user_rating_count is a groupby object that counts the number of ratings for each user.
t1b_user_rating_count = movielens_df.groupby('UserId')['Rating'].count()
#print t1b_user_rating_count
t1b_user_rating_count.hist(bins=50)
plt.xlabel("Ratings per user")
plt.ylabel("#Users")
plt.title("Count of Ratings per User")

####The figure below shows that most users leave less than 25 ratings while few outliers leave a lot of ratings

UserId
1      272
2       62
3       54
4       24
5      175
6      211
7      403
8       59
9       22
10     184
11     181
12      51
13     636
14      98
15     104
16     140
17      28
18     277
19      20
20      48
21     179
22     128
23     151
24      68
25      78
26     107
27      25
28      79
29      34
30      43
      ... 
914     23
915     26
916    317
917     35
918    103
919    217
920     26
921    110
922    127
923     74
924     82
925     32
926     20
927    120
928     32
929     49
930     63
931     61
932    241
933    184
934    174
935     39
936    142
937     40
938    108
939     49
940    107
941     22
942     79
943    168
Name: Rating, dtype: int64


In [24]:
#Task t1c: Let us now analyze the rating behavior of the 1000 users in the dataset
#  Create a histogram based on the number of ratings per movie with 50 bins. 
# Title="Count of Ratings per Movie", XLabel="Ratings per Movie", YLabel="#Movies"

#t1c_user_rating_count is a groupby object that counts the number of ratings for each movie.
t1c_user_rating_count = movielens_df.groupby('MovieId')['Rating'].count()
#print t1c_user_rating_count
t1c_user_rating_count.hist(bins=50)
plt.xlabel("Ratings per Movie")
plt.ylabel("#Movies")
plt.title("Count of Ratings per Movie")

####The figure below shows that most movies receive less than 25 ratings while few popular get a lot of ratings


MovieId
1       452
2       131
3        90
4       209
5        86
6        26
7       392
8       219
9       299
10       89
11      236
12      267
13      184
14      183
15      293
16       39
17       92
18       10
19       69
20       72
21       84
22      297
23      182
24      174
25      293
26       73
27       57
28      276
29      114
30       37
       ... 
1653      1
1654      1
1655      1
1656      2
1657      1
1658      3
1659      1
1660      1
1661      1
1662      2
1663      1
1664      4
1665      1
1666      1
1667      1
1668      1
1669      1
1670      1
1671      1
1672      2
1673      1
1674      1
1675      1
1676      1
1677      1
1678      1
1679      1
1680      1
1681      1
1682      1
Name: Rating, dtype: int64


In [26]:
#Task t1d: Let us now analyze the rating distribution
#  Create a histogram based on the ratings received for each movie with 5 bins. 
# Title="Ratings Histogram", XLabel="Rating Provided", YLabel="#Ratings"

movielens_df['Rating'].hist(bins=5)
plt.xlabel("Rating Provided")
plt.ylabel("#Ratings")
plt.title("Ratings Histogram")

####The figure below shows that most movies at least 35K of ratings provided a score of 4 or higher.
#The code below shows that the average rating is 3.5
# This is problematic because each user has a different rating scale
# For some users 1 is a bad movie while for some others 3 is bad
# So a good model must take into the account the baseline rating behavior of users and movies

print "Average rating of ALL movies is", round(movie_ratings_df['Rating'].mean(), 2)

Average rating of ALL movies is 3.53


In [31]:
#Task t1e: Let us now study the baseline rating behavior in more detail
# For each user compute his/her average rating
#  Create a histogram based on the average ratings with 5 bins. 
# Title="Histogram of Average Ratings of Users", XLabel="Average Rating", YLabel="#Users"

#t1e_avg_ratings is a groupby object with average rating for each user
t1e_avg_ratings = movielens_df.groupby('UserId')['Rating'].mean()
print t1e_avg_ratings

t1e_avg_ratings.hist(bins=5)
plt.title("Histogram of Average Ratings of Users")
plt.xlabel("Average Rating")
plt.ylabel("#Users")


####The figure below shows that while the average rating of users vary
# What does this imply?

UserId
1      3.610294
2      3.709677
3      2.796296
4      4.333333
5      2.874286
6      3.635071
7      3.965261
8      3.796610
9      4.272727
10     4.206522
11     3.464088
12     4.392157
13     3.097484
14     4.091837
15     2.875000
16     4.328571
17     3.035714
18     3.880866
19     3.550000
20     3.104167
21     2.670391
22     3.351562
23     3.635762
24     4.323529
25     4.051282
26     2.943925
27     3.240000
28     3.721519
29     3.647059
30     3.767442
         ...   
914    3.086957
915    3.115385
916    3.365931
917    3.542857
918    3.349515
919    3.470046
920    3.230769
921    3.272727
922    3.370079
923    4.148649
924    3.756098
925    3.125000
926    3.300000
927    3.691667
928    4.687500
929    3.693878
930    2.968254
931    3.721311
932    3.966805
933    2.646739
934    3.701149
935    3.923077
936    3.746479
937    3.375000
938    3.268519
939    4.265306
940    3.457944
941    4.045455
942    4.265823
943    3.410714
Name: Rating, dty

In [32]:
#Task t1f: Let us now study the baseline rating behavior in more detail
# For each movie compute its average rating
#  Create a histogram based on the average ratings with 5 bins. 
# Title="Histogram of Average Ratings of Movies", XLabel="Average Rating", YLabel="#Movies"

#t1e_avg_ratings is a groupby object with average rating for each user
t1f_avg_ratings = movielens_df.groupby('MovieId')['Rating'].mean()
#print t1f_avg_ratings

t1f_avg_ratings.hist(bins=5)
plt.title("Histogram of Average Ratings of Movies")
plt.xlabel("Average Rating")
plt.ylabel("#Movies")


####The figure below shows that while the average rating of movies vary
# What does this imply?

MovieId
1       3.878319
2       3.206107
3       3.033333
4       3.550239
5       3.302326
6       3.576923
7       3.798469
8       3.995434
9       3.896321
10      3.831461
11      3.847458
12      4.385768
13      3.418478
14      3.967213
15      3.778157
16      3.205128
17      3.119565
18      2.800000
19      3.956522
20      3.416667
21      2.761905
22      4.151515
23      4.120879
24      3.448276
25      3.443686
26      3.452055
27      3.105263
28      3.931159
29      2.666667
30      3.945946
          ...   
1653    5.000000
1654    1.000000
1655    2.000000
1656    3.500000
1657    3.000000
1658    3.000000
1659    1.000000
1660    2.000000
1661    1.000000
1662    2.500000
1663    2.000000
1664    3.250000
1665    2.000000
1666    2.000000
1667    3.000000
1668    3.000000
1669    2.000000
1670    3.000000
1671    1.000000
1672    2.000000
1673    3.000000
1674    4.000000
1675    3.000000
1676    2.000000
1677    3.000000
1678    1.000000
1679    3.000000
1680  

In [ ]:
#Task t1g: Let us now analyze the common support for movielens dataset
# Here is the high level idea
# We are going to create an array and populate it with the common support for all pair of movies
# We then are going to plot a histogram and see its distribution.

#This task might take quite some time - 1-2 hours for typical machines !

t1g_all_movies=movies_df['MovieId'].unique()
t1g_allpair_commonsupport=[]
'''
for i, movie1 in enumerate(t1g_all_movies):
    for j,movie2 in enumerate(t1g_all_movies):
        if(i<j):
            userids_who_rated_movie1=movies_df(movies_df.MovieId==movie1).UserId.unique()
            userids_who_rated_movie2=movies_df(movies_df.MovieId==movie2).UserId.unique()
            num_common_users=len(set(user_ids_who_rated_movie1).intersection(userids_who_rated_movie2))
            t1g_allpair_commonsupport.append(num_common_users)
            
print "Average common support is ", round(np.mean(t1g_allpair_commonsupport), 2)
plt.hist(t1g_allpair_commonsupport)

'''

In [ ]:
#Task t1h: Let us now consider how sparse the matrix is
t1h_sparsity = 100000./(1682*943)
print "Sparsity of the dataset is ", t1h_sparsity

#This graph is actually less sparse than typical datasets for recommender systems
# which often have a sparsity much lower than 1%
# As discussed in the class, the sparsity imposes huge problem in terms of designing efficient and correct algorithms

In [ ]:
#Task t1i: Compute the average rating for each movie grouped by gender
#  In other words, for each movie, compute the average rating given by men and women
# Hint: use a pivot table

t1i_movielens_mean_ratings = movielens_df.pivot_table('Rating', index ='Title', columns='Gender',aggfunc='mean')
display(t1i_movielens_mean_ratings[:10])


In [36]:
#Nearest Neighbour based Recommender System
#Task t2a: 
# Create a dictionary where key is Movie Name and value is id
#       You can either use the movies_df or read and parse the u.item file yourself
movie_name_to_id_dictionary = {}

#Write code to populate the movie names to this array
all_movie_names = []

f = open("ml-100k/u.item", "r")
lines = f.readlines()
for line in lines:
    data = line.split("|")
    movie_name, movie_id = data[1], int(data[0])
    movie_name_to_id_dictionary[movie_name] = movie_id
    all_movie_names.append(movie_name)
f.close()

#print movie_name_to_id_dictionary
#print all_movie_names

{'Adventures of Pinocchio, The (1996)': 1060, 'To Cross the Rubicon (1991)': 1564, 'Birdcage, The (1996)': 25, 'B. Monkey (1998)': 1679, 'Little Big League (1994)': 1032, 'Magnificent Seven, The (1954)': 510, 'Chairman of the Board (1998)': 1654, 'Mortal Kombat (1995)': 541, 'Tin Men (1987)': 712, 'Endless Summer 2, The (1994)': 1184, 'Village of the Damned (1995)': 565, 'Cat on a Hot Tin Roof (1958)': 499, 'SubUrbia (1997)': 1428, 'Air Bud (1997)': 261, 'Johnny 100 Pesos (1993)': 1365, 'Bye Bye, Love (1995)': 1446, 'Men With Guns (1997)': 1646, 'Beautiful Thing (1996)': 1137, '2 Days in the Valley (1996)': 1011, 'Jerky Boys, The (1994)': 1484, 'Close Shave, A (1995)': 408, 'Amadeus (1984)': 191, 'Critical Care (1997)': 341, 'Clockwork Orange, A (1971)': 179, 'Country Life (1994)': 1290, 'Boys in Venice (1996)': 1359, 'Wings of Courage (1995)': 1515, 'Fifth Element, The (1997)': 250, 'Heaven & Earth (1993)': 803, 'Tombstone (1993)': 470, 'Lawnmower Man 2: Beyond Cyberspace (1996)': 758

In [43]:
#Task t2b: Write a function that takes two inputs: 
#  movie_id: id of the movie and common_users: a set of user ids
# and returns the list of rows corresponding to their movie ratings 

def get_movie_reviews(movie_id, common_users):
    #Get a boolean vector for themovielens_dfns_dfs provided by users in common_users for movie movie_id
    # Hint: use the isin operator of Pandas
    mask = (movielens_df['UserId'].isin(common_users)) & (movielens_df['MovieId'] == movie_id)
    
    
    #Create a subset of data where the mask is True i.e. only collect data from users who satisfy the condition above
    # Then sort them based on userid
    movie_ratings = movielens_df[mask].sort('UserId')
    #print movie_ratings
    
    #Do not change below
    #Return the unique set of ratings provided
    movie_ratings = movie_ratings[movie_ratings['UserId'].duplicated()==False]
    return movie_ratings


get_movie_reviews(1, set(range(1, 10)))

0         True
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
99970    False
99971    False
99972    False
99973    False
99974    False
99975    False
99976    False
99977    False
99978    False
99979    False
99980    False
99981    False
99982    False
99983    False
99984    False
99985    False
99986    False
99987    False
99988    False
99989    False
99990    False
99991    False
99992    False
99993    False
99994    False
99995    False
99996    False
99997    False
99998    False
99999    False
Name: MovieId, dtype: bool


,MovieId,Title,UserId,Rating,Age,Gender,Occupation,ZipCode
20103,1,Toy Story (1995),1,5,24,M,technician,85711
15002,1,Toy Story (1995),2,4,53,F,other,94043
820,1,Toy Story (1995),5,4,33,F,other,15213
35865,1,Toy Story (1995),6,4,42,M,executive,98101


In [ ]:
#Do not change below

#Here are some sample test cases for evaluating t2b
print "get_movie_reviews(1, set([1]))"
display( get_movie_reviews(1, set([1])) )

print "get_movie_reviews(1, set(range(1, 10)))"
display( get_movie_reviews(1, set(range(1, 10))) )

print "get_movie_reviews(100, set(range(1, 10)))"
display( get_movie_reviews(100, set(range(1, 10))) )

print "get_movie_reviews(784, set(range(1, 784)))"
display( get_movie_reviews(784, set(range(1, 784))) 

In [55]:
#Task t2c: Let us now calculate the similarity between two movies
# based on the set of users who rated both movies
#Using euclidean distance is a bad idea - but simplifies the code

def calculate_similarity(movie_name_1, movie_name_2, min_common_users=0):
    
    movie1 = movie_name_to_id_dictionary[movie_name_1]
    movie2 = movie_name_to_id_dictionary[movie_name_2]
    
    #This is the set of UNIQUE user ids  who reviewed  movie1
    users_who_rated_movie1 = movielens_df[movielens_df['MovieId'] == movie1]['UserId'].unique()
    
    #This is the set of UNIQUE user ids  who reviewed  movie2
    users_who_rated_movie2 = movielens_df[movielens_df['MovieId'] == movie2]['UserId'].unique()
    
    #Compute the common users who rated both movies: 
    # hint convert both to set and do the intersection
    common_users = set(users_who_rated_movie1).intersection(users_who_rated_movie2)
    
    #Using the code you wrote in t2a, get the reviews for the movies and common users
    movie1_reviews = get_movie_reviews(movie1, common_users)
    movie2_reviews = get_movie_reviews(movie2, common_users)
    
    #Now you have the data frame for both movies
    # Use the euclidean_distances function from sklean (imported already)
    # to compute the distance between their rating values
    distance = euclidean_distances(movie1_reviews['Rating'], movie2_reviews['Rating'])

    if len(common_users) < min_common_users:
        return [[float('inf')]]
    return distance


In [54]:
#Do not change below
print calculate_similarity("Toy Story (1995)", "GoldenEye (1995)")
print calculate_similarity("GoldenEye (1995)", "Tomorrow Never Dies (1997)")
print calculate_similarity("Batman Forever (1995)", "Batman & Robin (1997)")

[308 287 148 280  66   5 109 181  95 268 189 145 158  67 232 150 289 117
  49 223  56  17 340 177 194 250 213 350 348 131 106 234  43  20 246  94
 279  38 128  96 203 157 311 125 286  83 301 345  18 322  64  45 247 204
 271  41 274 222  15  23 141 359 374 380  10 320 160 396 339 199 422  81
  42 357  26 327 200  44 256 174  97 243 101 490 242 312 325 456 347 360
 450  84 198 494 525   2  62 343 298 459 407 471 535 500 512 275 252 230
 249  93 514 193 484  54  75 338  13 472 541 209 144 417 454 313 202 579
 447 137 508   1 569 416 291 401 479 567 561 390 162 332 363 455 394 120
 297 540 231 560 379  59  99 254 533 441 292 262 483  16 592 526 307 344
 649 299 324 545 517  82 658 460 622 429  73 326 330 532 470 620 263 184
 577 248 699 534 505 425 691 406 432 657 305 487  77 235 151 642 708 244
 735 523 749 705 636 276 635 478 381  92 467 716 763 465 613 588 676 643
 562 178 677 253 697 614 798 777 637 371 290 770 618 684  89 706   6  63
 674 648  65 497 731 378 655 395 835 768 402 435 13

TypeError: object of type 'NoneType' has no len()

In [ ]:


#Task t2d: Given a movie, find the top-k most similar movies
# that have the lowest euclidean distance 

#Here is the high level logic:
#  for each movie in all_movie_names (Except the input movie name)
#    compute its similarity and store it in an array
#   return the k movies with the smallest distances
# remember to pass min_common_users to calculate_similarity
def get_top_k_similar_movies(input_movie_name, k=5, min_common_users=0):
    movie_similarity = [] 
    for movie_name in all_movie_names:
        if input_movie_name == movie_name:
            continue
        similarity = calculate_similarity(input_movie_name, movie_name, min_common_users)
        similarity = similarity[0][0]
        movie_similarity.append( (similarity, movie_name) )
    return sorted(movie_similarity)[:k]

In [ ]:
#print get_top_k_similar_movies("Toy Story (1995)", 10)
print "\nMovies similar to GoldenEye [25]", get_top_k_similar_movies("GoldenEye (1995)", 10, 25)
print "\nMovies similar to GoldenEye [50]", get_top_k_similar_movies("GoldenEye (1995)", 10, 50)
print "\nMovies similar to GoldenEye [100]", get_top_k_similar_movies("GoldenEye (1995)", 10, 100)
print "\n\n"

print "\nMovies similar to Usual Suspects [25]", get_top_k_similar_movies("Usual Suspects, The (1995)", 10, 25)
print "\nMovies similar to Usual Suspects [50]", get_top_k_similar_movies("Usual Suspects, The (1995)", 10, 50)
print "\nMovies similar to Usual Suspects [100]", get_top_k_similar_movies("Usual Suspects, The (1995)", 10, 100)
print "\n\n"

print "\nMovies similar to Batman Forever [25]", get_top_k_similar_movies("Batman Forever (1995)", 10, 25)
print "\nMovies similar to Batman Forever [50]", get_top_k_similar_movies("Batman Forever (1995)", 10, 50)
print "\nMovies similar to Batman Forever [100]", get_top_k_similar_movies("Batman Forever (1995)", 10, 100)
print "\n\n"

print "\nMovies similar to Shawshank Redemption [25]", get_top_k_similar_movies("Shawshank Redemption, The (1994)", 10, 25)
print "\nMovies similar to Shawshank Redemption [50]", get_top_k_similar_movies("Shawshank Redemption, The (1994)", 10, 50)
print "\nMovies similar to Shawshank Redemption [100]", get_top_k_similar_movies("Shawshank Redemption, The (1994)", 10, 100)
print "\n\n"

In [ ]:
#Item based Collabrative Filtering
#Do not change below
# By default euclidean distance can be give arbitrary values
# Let us "normalize" it by limit its value between 0 and 1 and slightly change the interpretation
# 0 means that the preferences are very different
# 1 means that preferences are identical
# For tasks 3 and 4, remember to use this function

#Vec1 and vec2 are vectors
def euclidean_distance_normed(vec1, vec2):
    if len(vec1) == 0:
        return 0.0
    euc_distance = euclidean_distances(vec1, vec2)[0][0]
    return 1.0 / (1.0 + euc_distance)

In [ ]:
#Task t3a:
# In this task, you want to compute the similarity between two items
#  which in this case means ratings_df
# You can use code from 2c except that you must now call euclidean_distance_normed
#  when computing the distance

def calculate_similarity_normed(movie_name_1, movie_name_2, min_common_users=0):
    movie1 = movie_name_to_id_dictionary[movie_name_1]
    movie2 = movie_name_to_id_dictionary[movie_name_2]

    #This is the set of UNIQUE user ids  who reviewed  movie1
    users_who_rated_movie1 = movielens_df[movielens_df['MovieId'] == movie1]['UserId'].unique()
    
    #This is the set of UNIQUE user ids  who reviewed  movie2
    users_who_rated_movie2 = movielens_df[movielens_df['MovieId'] == movie2]['UserId'].unique()
     
    #Compute the common users who rated both movies: 
    # hint convert both to set and do the intersection
    common_users = set(users_who_rated_movie1).intersection(users_who_rated_movie2)

    #Using the code you wrote in t2a, get the reviews for the movies and common users
    movie1_reviews = get_movie_reviews(movie1, common_users)
    movie2_reviews = get_movie_reviews(movie2, common_users)
    
    #Do not change below
    
    #Now you have the data frame for both movies
    # Use the euclidean_distances function from sklean (imported already)
    # to compute the distance between their rating values
    distance = euclidean_distance_normed(movie1_reviews['Rating'].values, movie2_reviews['Rating'].values)

    if len(common_users) < min_common_users:
        return 0.0
    return distance

In [ ]:
#Do not change below
print calculate_similarity_normed("Toy Story (1995)", "GoldenEye (1995)")
print calculate_similarity_normed("GoldenEye (1995)", "Tomorrow Never Dies (1997)")
print calculate_similarity_normed("Batman Forever (1995)", "Batman & Robin (1997)")

In [ ]:
#Do not change below

#We are now going to create item-item similarity database
# Since our data is "small", we will use a non-traditional approach of using nested hashes
# In real-life, using something like databases or other data structures is far more preferable

#Here is the high level structure
#{
#    movie_name_1:  
#    { 
#        movie_name_2: similarity_between_movie_1_and_2, 
#        movie_name_3: similarity_between_movie_1_and_3, 
#        ....
#        movie_name_n: similarity_between_movie_1_and_n
#    },
#    movie_name_2:
#    {
#        movie_name_1: similarity_between_movie_2_and_1, 
#        movie_name_3: similarity_between_movie_2_and_3, 
#        ....
#        movie_name_n: similarity_between_movie_2_and_n
#    },
#    ....
#    movie_name_n:
#    {
#        movie_name_1: similarity_between_movie_n_and_1, 
#        movie_name_2: similarity_between_movie_n_and_2, 
#        ....
#        movie_name_n-1: similarity_between_movie_n_and_n-1
#    },
#}    
    

#Here is how to use this data structuere:

#To get similarity between movies
#   data[movie1][movie2]
#To get similarity between one movie and all others
#   data[movie1]

In [ ]:
#DO not change below
#This hash stores the movie to movie 
# as described above
movie_similarity_hash = defaultdict(dict)

In [2]:


#Item based filtering is expensive as you need to compute similarity of all pairs of items
# for this dataset it is 1682*1682 ~ 28 lakh pairs or 2.8 million
# running all of them might take hours and close to a day
# instead let us run on a smaller dataset
# specifically, let us only focus on the top-250 movies based on ratings
# which is more manageable


#Task t3b: 
# Get the top-k movie names with most ratings
#Hint: use Counter class

def top_k_movie_names(k):
    movie_ratings_counter = Counter()
    
    with open("ml-100k/u.data","r") as f:
        for line in f:
            user_id, movie_id, rating, timestamp = line.split("\t")
            movie_name = all_movie_names[ int(movie_id) - 1]
            movie_ratings_counter[movie_name] += 1
    
    return movie_ratings_counter.most_common(k)

#Do not change below
print "Top-10", top_k_movie_names(10), "\n"
print "Top-25", top_k_movie_names(25), "\n"

Top-10

NameError: global name 'Counter' is not defined

In [1]:
#Do not change below
top_250_movie_names = [item[0] for item in top_k_movie_names(250)]

NameError: name 'top_k_movie_names' is not defined

In [ ]:
#Task t3c:
#Use the following logic
#  for each movie in movie_names:
#    for all other movies in movie_names:
#      compute similarity between  two movies using calculate_similarity_normed
#      remember to pass min_common_users to that function
#  note that movie_similarity_hash is a defaultdict 
#  so similarity between movie1 and movie2 can be set as movie_similarity_hash[movie1][movie2]
#  btw, similarity in our case is commutative. 
#   i.e. similarity(movie1, movie2) = similarity(movie2, movie1)
#   so do not call the function twice !
# movie_names is an array that lists the movies for which you have to compute pairwise similarity
def compute_movie_to_movie_similarity(movie_names, min_common_users=0):
    for movie_name_1 in movie_names:
        for movie_name_2 in movie_names:
            if movie_name_1 == movie_name_2:
                continue
            similarity = calculate_similarity_normed(movie_name_1, movie_name_2, min_common_users)
            movie_similarity_hash[movie_name_1][movie_name_2] = similarity
            movie_similarity_hash[movie_name_2][movie_name_1] = similarit

In [ ]:
#Do not change below

#Let us first test if your code above is correct by testing against a small subset of data
movie_similarity_hash = defaultdict(dict)
# let use the top-10 movies
compute_movie_to_movie_similarity(top_250_movie_names[:10], min_common_users=0)

#Get similarity with 
display(movie_similarity_hash["Toy Story (1995)"])
display(movie_similarity_hash['Return of the Jedi (1983)'])

print movie_similarity_hash["Toy Story (1995)"]["Independence Day (ID4) (1996)"]

In [ ]:
#Do not change below
#Let us now test against top-250 most popular movies
#This might take 10-20 mins to run!
movie_similarity_hash = defaultdict(dict)
compute_movie_to_movie_similarity(top_250_movie_names, min_common_users=25)

In [ ]:
#Do not change below
#Do this if you want to persist the data 

# Let us persist the movie-movie similarity data structure 
# that way you dont need to re-run the whole thing
#pickle is a serialization library in Python
# To persist/serialize, use the following line
#pickle.dump(movie_similarity_hash, open("movie_similarity.pickle", "wb"))
# To deserialize, uncomment the following line 
#movie_similarity_hash = pickle.load( open( "movie_similarity.pickle", "rb" ) )

In [ ]:
for movie_name in top_250_movie_names[:10]:
    print "Top-10 most similar movies for ", movie_name, " :", 
    print sorted(movie_similarity_hash[movie_name].items(), key=operator.itemgetter(1), reverse=True)[:10]
    print "\n"

NameError: global name 'all_movie_names' is not defined

In [ ]:
#Task t3d

#Before doing t3d, please complete t4a so that user_rating_hash is available
# this will make the code below easier

#In this task, we are going to predict the rating of a user u for a movie m using item based collaborative filtering
#Here is the high level logic:
# for each item i rated by this user:
#    s = similarity between i and input movie m 
#    if similarity between i and m is 0, ignore this item 
#    compute weighted rating for m based on i as rating for i * s
# compute the predicted rating as sum of all weighted ratings / sum of all similarities

def predict_rating_for_movie_icf(movie_similarity_hash, input_user_id, input_movie_name, movies_considered):
    total_weighted_rating = 0.0
    total_similarity= 0.0
        
    #Hint: movie_similarity_hash is a nested hash where user id is key and 
    #  all their rating as a dictionary  
    # this dictionary is ordered as moviename: rating

    #if this user has already rated the movie, return that rating
    if input_movie_name in user_rating_hash[input_user_id]:
        return user_rating_hash[input_user_id][input_movie_name]
    
    #For each movie the user has rated
    for movie_name in user_rating_hash[input_user_id].keys():

        #if user rated some movie, but it is not in the subset of movies that we computed pairwise similarity
        # such as top-250, then do not consider it either
        # for this task, the input is in movies_considered 
        if movie_name not in movies_considered:
            continue
        
        #compute similarity between movies
        #dont recompute = use the hash
        similarity = movie_similarity_hash[input_movie_name][movie_name]
        
        #Reject item if similarity is 0
        if similarity <= 0.0:
            continue
                     
        #Compute weighted rating
        weighted_rating = similarity * user_rating_hash[input_user_id][movie_name]
        
        #update total_weighted_rating and total_similarity
        total_weighted_rating += weighted_rating
        total_similarity += similarity
            
    #Do not change below
    if total_similarity == 0.0:
        return 0.0
    
    return total_weighted_rating / total_similarity

In [ ]:
#Task t3e: 
#Here is the pseudocode for recommending movies
# for each movie this user has not rated in movies_considered:
#           predict rating for this movie and this user using t4d
#  return the top-k movies
def recommend_movies_icf(input_user_id, movies_considered, movie_similarity_hash,
                             user_rating_hash, k=10, min_common_movies=5):
    predicted_ratings = []
    
    #Your code here
    for movie_name in movies_considered:
        if movie_name in user_rating_hash[input_user_id]:
            continue
        
        #Predict the rating for input_user_id and movie_name using t3d
        predicted_rating = predict_rating_for_movie_icf(movie_similarity_hash, 
                                        input_user_id, movie_name, movies_considered)
        #append rating and movie name to predicted_ratings
        predicted_ratings.append( (predicted_rating, movie_name) )
    
    return sorted(predicted_ratings, reverse=True)[:k]

In [ ]:
#Do not change below:

#Let us predict top-5 movies for first 10 users
for user_id in range(1,11):
    print user_id, recommend_movies_icf(user_id, top_250_movie_names, movie_similarity_hash, 
                               user_rating_hash, k=10, min_common_movies=5)

In [ ]:
# User based Collabrative Filtering



#Task t4a
#Create the data structure as discussed above
# here is the logic:
# for each line in file ml-100k/u.data:
#   set user_rating_hash[user][movie] = rating
# read the instructions above again!

def compute_user_rating_hash():
    user_rating_hash = defaultdict(dict)
    
    #Your code below
    with open("ml-100k/u.data","r") as f:
        for line in f:
            user_id, movie_id, rating, timestamp = line.split("\t")
            user_rating_hash[int(user_id)][all_movie_names[int(movie_id)-1]] = int(rating)    
    
    return user_rating_hash



In [ ]:
#Do not change below
user_rating_hash = compute_user_rating_hash()

In [ ]:
#Do not change below
#How many users are there?
print len(user_rating_hash.keys())
#How many movies did each of the first 20 users rated?
print [len(user_rating_hash[i].keys()) for i in range(1,20+1)] 
#print the ratings of user 4
display(user_rating_hash[4])

In [ ]:
#Task t4b:
#We need to modify our logic for computing distance
#Here is the high level pseudocode:
# movie1 = movie names rated by user 1
# movie2 = movie names rated by user 2
# common movies = set intersection of movie1 and movie2
# if number of common movies is less than min_common_movies, return 0.0 [not 0]
# other wise create a vector with rating for common movies only
# compute euclidean distance between the vectors
# return 1 / (1+euclidean distace)

def compute_user_user_similarity(user_rating_hash, user_id_1, user_id_2, min_common_movies=0):
    #Get list of movie names rated by user 1. hint use keys function [see above for usage]
    movies_rated_user1 = user_rating_hash[user_id_1].keys()
    movies_rated_user2 = user_rating_hash[user_id_2].keys()
    
    #compute common movies
    common_movies = set(movies_rated_user1).intersection( set(movies_rated_user2) )
    if len(common_movies) < min_common_movies:
        return 0.0
    
    common_movies = sorted(list(common_movies))
    
    #vector1 is the set of ratings for user1 for movies in common_movies
    vector1 = [user_rating_hash[user_id_1][movie] for movie in common_movies]
    #vector2 is the set of ratings for user2 for movies in common_movies
    vector2 = [user_rating_hash[user_id_2][movie] for movie in common_movies]
    
    #Compute distance and return 1.0/(1.0+distance)
    distance = euclidean_distances(vector1, vector2)[0][0]
    return 1.0 / ( 1.0 + distance)

In [ ]:
#Testing code
print [round(compute_user_user_similarity(user_rating_hash, 1, i),2) for i in range(1, 10+1)]
print [round(compute_user_user_similarity(user_rating_hash, 784, i),2) for i in range(1, 10+1)]


In [ ]:
#Task t4c
#This function finds the k-most similar users 
#Here is the high level logic:
#  for each user in all_user_ids other than the input user id:
#     find similarity between this user and input_user_id and store as (similarity, other userid)
#     sort based on similarity
#  return top-k
# remember to pass min_common_movies
def top_k_most_similar_users(user_rating_hash, input_user_id, all_user_ids, k=10, min_common_movies=0):
    user_similarity = []
    
    for user_id in all_user_ids:
        if user_id == input_user_id:
            continue
        similarity = compute_user_user_similarity(user_rating_hash, input_user_id, user_id, min_common_movies)
        user_similarity.append( ( similarity, user_id) )
    return sorted(user_similarity, reverse=True)[:k]

In [ ]:
#Do not change below
all_user_ids = range(1, 943+1)
print top_k_most_similar_users(user_rating_hash, 1, all_user_ids, 10, 5)
print top_k_most_similar_users(user_rating_hash, 1, all_user_ids, 10, 10)
print top_k_most_similar_users(user_rating_hash, 812, all_user_ids, 10, 5)
print top_k_most_similar_users(user_rating_hash, 812, all_user_ids, 10, 20)


In [ ]:
#Task t4d
#In this task, we are going to predict the rating of a user for a movie using user based collaborative filtering
#Here is the high level logic:
# for each user u in all_user_ids:
#    s= similarity between u and input_user_id [remember to pass min_common_movies]
#    if similairty is 0.0 ignore u
#    if u has not rated this movie, ignore again
#    suppose u has rated this movie with a value of r
#    i am now going to give a "weighted rating" as r*s
# compute the predicted rating as sum of all weighted ratings / sum of all similarities

def predict_rating_for_movie_ucf(user_rating_hash, input_user_id, movie_name, all_user_ids, min_common_movies=5):
    total_weighted_rating = 0.0
    total_similarity= 0.0
    
    for user_id in all_user_ids:
        if user_id == input_user_id:
            continue
        
        #compute similarity between users
        similarity = compute_user_user_similarity(user_rating_hash, user_id, input_user_id, min_common_movies)
        
        #Reject user if similarity is 0
        if similarity <= 0.0:
            continue
            
        #reject user if (s)he has not rated the movie
        if movie_name not in user_rating_hash[user_id]:
            continue
            
        #Compute weighted rating
        weighted_rating = similarity * user_rating_hash[user_id][movie_name]
        
        #update total_weighted_rating and total_similarity
        total_weighted_rating += weighted_rating
        total_similarity += similarity
            
    #Do not change below
    if total_similarity == 0.0:
        return 0.0
    
    return total_weighted_rating / total_similarity

In [ ]:


#Do not change below
all_user_ids = range(1, 943+1)
for user_id in range(1, 5+1):
    print "user_id = ", user_id
    print [ round(predict_rating_for_movie_ucf(user_rating_hash, user_id, all_movie_names[i], all_user_ids, min_common_movies=5),1)
          for i in range(1, 10+1)]
    print [ round(predict_rating_for_movie_ucf(user_rating_hash, user_id, all_movie_names[i], all_user_ids, min_common_movies=10),1)
          for i in range(1, 10+1)]
    print "\n"



In [ ]:
#Task t4e: 
#Here is the pseudocode for recommending movies
# for each movie this user has not rated:
#     for all other users:
#           predict rating for this movie and this user using t4d
#  return the top-k movies
def recommend_movies_ucf(user_rating_hash, all_user_ids, input_user_id, k=10, min_common_movies=5):
    predicted_ratings = []
    
    #Your code here
    for movie_name in all_movie_names:
        if movie_name in user_rating_hash[input_user_id]:
            continue
        
        #Predict the rating for input_user_id and movie_name using t4d
        predicted_rating = predict_rating_for_movie_ucf(user_rating_hash, input_user_id, movie_name, 
                                                        all_user_ids, min_common_movies)
        #append rating and movie name to predicted_ratings
        predicted_ratings.append( (predicted_rating, movie_name) )
    
    return sorted(predicted_ratings, reverse=True)[:k

In [ ]:
#Do not change below
all_user_ids = range(1, 943+1)

for user_id in range(1, 5):
    print recommend_movies_ucf(user_rating_hash, all_user_ids, user_id, k=10, min_common_movies=5)